In [1]:
# Summarize Emission Totals from Soundcast Output

In [1]:
import pandas as pd
import os

In [2]:
run_dir = r'L:\vision2050\soundcast\integrated\stc\2050'
df_summer = pd.read_csv(os.path.join(run_dir, r'outputs\aq_2050_july.csv'))
df_winter = pd.read_csv(os.path.join(run_dir, r'outputs\aq_2050_jan.csv'))

In [3]:
def grams_to_tons(value):
   
    value = value/453.592
    value = value/2000
    
    return value

In [28]:
#### Pollutant codes
# https://github.com/CEMPD/SMOKE-MOVES/wiki/Runspec-generator-pollutant-options

# All pollutants to be evaluated for winter, with the exception of those listed in summer_list

# 1: total gaseous HCs
# 2: CO
# 3: NOx
# 5: Methane
# 6: N20
# 79: Non-methane HCs
# 87: VOCs                 # Summer
# 90: atmospheric CO2
# 98: CO2 equiv

#### PM10
# 100: PM10 exhaust
# 106: PM10 brakewear
# 107: PM10 Tirewear

#### PM2.5
# 110: PM2.5 exhaust
# 116: PM2.5 brakewear
# 117: PM2.5 tirewear

pollutant_list = [str(i) for i in [1,2,3,5,6,79,87,90,98,100,106,107,110,116,117]]

 # Pollutants to be evaluated in summer rather than winter
summer_list = [87]   

# Running Emissions

In [29]:
# Initialize results dictionary
running_emissions = {k: 0 for k in pollutant_list}

# Loop through each pollutant to calculate totals
for pollutant in pollutant_list:
    # Load rates based on analysis season
    if pollutant in summer_list:
        df = df_summer.copy()
    else:
        df = df_winter.copy()
    df['tot'] = df['length']*df['total_volume']*df[pollutant]
    tot = df['tot'].sum()
    running_emissions[pollutant] += grams_to_tons(tot)

In [30]:
# Calculate total PM10 and PM2.5
running_emissions['PM10'] = running_emissions['100']+running_emissions['106']+running_emissions['107']
running_emissions['PM25'] = running_emissions['110']+running_emissions['116']+running_emissions['117']

In [31]:
df_running = pd.DataFrame.from_dict(running_emissions, orient='index').reset_index()
df_running.columns = ['pollutant','running_tons']

# Intrazonals

In [32]:
# Load emissions rate table to calculate intrazonal trips
rates = pd.read_csv(os.path.join(run_dir,r'scripts\summarize\inputs\network_summary\emission_rates_2040.csv'))

In [33]:
tod_lookup = {'5to6' : 5, '6to7' : 6, '7to8' : 7, '8to9' : 8, '9to10' : 9, 
              '10to14' : 10, '14to15' : 14, '15to16' : 15, '16to17' : 16, 
              '17to18' : 17, '18to20' : 18, '20to5' : 20}

county_id = {1: 'King',
            2: 'Kitsap',
            3: 'Pierce',
            4: 'Snohomish'}

In [34]:
# Intrazonal volumes 
# From emissions rates table, look up factor by time of day
# Use assumed speed bin and roadway type
speedbin = 4
roadtype = 5

# Initialize dictionary of intrazonal (iz) emissions totals
iz_emissions = {k: 0 for k in pollutant_list}

# Loop through each pollutant
for pollutant in pollutant_list:
    print pollutant
    for tod in tod_lookup.keys():
        if pollutant in summer_list:
            month = 7
        else:
            month = 1
            
        # Load total intrazonal VMT from soundcast outputs
        df_iz = pd.read_excel(os.path.join(run_dir,r'outputs\aq_2050.xlsx'), sheetname=tod+'_intrazonal')
        df_iz['tot_vmt'] = df_iz['sov_iz_vmt']+df_iz['hov2_iz_vmt']+df_iz['hov3_iz_vmt']+df_iz['med_truck_iz_vmt']+df_iz['hvy_truck_iz_vmt']

        # Filter rates for given roadtype, speed, month, pollutant, and TOD
        df = rates[(rates['roadtypeId'] == roadtype) & 
                   (rates['avgspeedbinId'] == speedbin) & 
                   (rates['monthId'] == month) & 
                   (rates['pollutantId'] == int(pollutant)) &
                   (rates['hourId'] == tod_lookup[tod])]

        # Map county ID to name to match rates and soundcast output
        df['geog_name'] = df['countyId'].map(county_id)

        # Join total intrazonal VMT with emissions rates
        df = pd.merge(df_iz[['tot_vmt','geog_name']],df[['geog_name','gramsPerMile']],on='geog_name')
        df['emissions_total'] = df['tot_vmt']*df['gramsPerMile']

        # Sum across time period by pollutant
        iz_emissions[pollutant] += grams_to_tons(df['emissions_total'].sum())

1
2
3
5
6
79
87
90
98
100
106
107
110
116
117


In [35]:
# Calculate total PM10 and PM2.5
iz_emissions['PM10'] = iz_emissions['100']+iz_emissions['106']+iz_emissions['107']
iz_emissions['PM25'] = iz_emissions['110']+iz_emissions['116']+iz_emissions['117']

In [36]:
df_iz = pd.DataFrame.from_dict(iz_emissions, orient='index').reset_index()
df_iz.columns = ['pollutant','intrazonal_tons']

# Starting Emissions

In [37]:
# Import rate per vehicle
starts = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\starts_king_2040.csv')

# using wintertime for all emission rates per X:\Trans\AIRQUAL\T2040 2018 Update\EmissionCalcs\Start Emissions\Starts_2040.xlsx
# Sum across processID and hourID
month = 1
grams_per_veh = starts[starts['monthID'] == month].groupby('pollutantID').sum()[['ratePerVehicle']]

In [38]:
# Vehicle population
veh_pop = {'King': 2038221,
          'Kitsap': 289947,
          'Pierce': 847228,
          'Snohomish': 807183}

# Adjust this for 2050


# Calculate total grams per day
veh_pop = pd.DataFrame.from_dict(veh_pop,orient='index').reset_index()
veh_pop.columns = ['geog_name','vehicles']

In [39]:
vehs = veh_pop[veh_pop['geog_name'] == 'King']['vehicles'].values[0]
total_grams = grams_per_veh['ratePerVehicle'] * vehs

In [40]:
start_tons = pd.DataFrame(grams_to_tons(total_grams)).reset_index()
start_tons.columns = ['pollutant','start_tons']
start_tons['pollutant'] = start_tons['pollutant'].astype('str')

# PM 10 and 2.5 only available for 100 and 110 for starts, respectively
# Rename these pollutants to match other totals
index_map = {k: k for k in start_tons['pollutant'].values} 
index_map['100'] = 'PM10'
index_map['110'] = 'PM25'
start_tons['pollutant'] = start_tons['pollutant'].map(index_map)

In [41]:
start_tons

,pollutant,start_tons
0,1,6.236982
1,2,65.826914
2,3,3.860276
3,5,0.558763
4,6,0.320829
5,79,5.678215
6,87,5.826669
7,90,1385.606795
8,91,0.018259
9,98,1495.184329


# Total Daily Emissions

In [42]:
# Combine link, intrazonal, and start emissions

df_daily = pd.merge(df_iz,df_running,how='left').fillna(0)
df_daily = pd.merge(df_daily,start_tons,how='left').fillna(0)
df_daily['daily_tons'] = df_daily['intrazonal_tons'] + df_daily['running_tons'] + df_daily['start_tons']

# Sort by pollutant id, requies some trickery
df_a = df_daily[(df_daily['pollutant'] != 'PM10') & (df_daily['pollutant'] != 'PM25')]
df_a['pollutant'] = df_a['pollutant'].astype('int')
df_a = df_a.sort_values('pollutant')
df_a['pollutant'] = df_a['pollutant'].astype('str')
df_b = df_daily[-((df_daily['pollutant'] != 'PM10') & (df_daily['pollutant'] != 'PM25'))]

df_daily = pd.concat([df_a,df_b], sort=True)

# Map pollutant name and ID
pollutant_map = {
    '1': 'Total Gaseous HCs',
    '2': 'CO',
    '3': 'NOx',
    '5': 'Methane',
    '6': 'N20',
    '79': 'Non-methane HCs',
    '87': 'VOCs',             
    '90': 'Atmospheric CO2',
    '98': 'CO2 Equivalent',
    'PM10': 'PM10 Total',
    'PM25': 'PM25 Total',
    '100': 'PM10 Exhaust',
    '106': 'PM10 Brakewear',
    '107': 'PM10 Tirewear',
    '110': 'PM25 Exhaust',
    '116': 'PM25 Brakewear',
    '117': 'PM25 Tirewear',   
}

df_daily['pollutant_name'] = df_daily['pollutant'].map(pollutant_map)

In [43]:
df_daily

,daily_tons,intrazonal_tons,pollutant,running_tons,start_tons,pollutant_name
4,8.783060,0.017321,1,2.528757,6.236982,Total Gaseous HCs
7,181.647974,0.687692,2,115.133368,65.826914,CO
6,20.753477,0.090884,3,16.802317,3.860276,NOx
8,1.151567,0.003994,5,0.588809,0.558763,Methane
11,0.467922,0.001321,6,0.145772,0.320829,N20
16,7.388278,0.011553,79,1.698511,5.678215,Non-methane HCs
15,7.544446,0.011727,87,1.706050,5.826669,VOCs
12,40775.906445,248.286860,90,39142.012790,1385.606795,Atmospheric CO2
5,40944.134996,248.780190,98,39200.170478,1495.184329,CO2 Equivalent
13,0.599684,0.003612,100,0.596071,0.000000,PM10 Exhaust
